# UP Compilers Example

In this example we will create a unified_planning problem and then show how to use a compiler on it to obtain a new equivalent problem; then, we will get a plan for the compiled problem and translate it into an equivalent plan for the original problem.

## Setup the UP library

We start by installing the library with PIP

In [ ]:
# begin of installation

In [ ]:
!pip install --pre unified-planning

In [ ]:
# end of installation

## Define the UP Problem

For this example we will create a problem with the following features:
- default temperature is cold
- 2 jobs that can be done only if it is warm
- 2 heaters with some quirks:   
    1) If both heaters are switched on at the same time, it will cause an electrical failure.  
    2) Once an heater is switched on, the heat it provides can be used only for one job, after that the heater will not provide heat anymore.  
    3) Every heater can be switched on only once.

In the end we want every job done, no heaters switched on and no electrical failure.

In [ ]:
from unified_planning.shortcuts import *

# Define User Types
Heater = UserType("Heater")
Job = UserType("Job")
Clean = UserType("Clean", Job)
Work = UserType("Work", Job)

# Define fluents
is_cold = Fluent("is_cold", BoolType()) # BoolType is the default, so it can be avoided
is_warm = Fluent("is_warm")
electrical_failure = Fluent("electrical_failure")
job_done = Fluent("job_done", BoolType(), job = Job)
is_on = Fluent("is_on", BoolType(), heater = Heater)
used_heater = Fluent("used_heater", BoolType(), heater = Heater)

# Define actions
switch_heater_on = InstantaneousAction("switch_heater_on", heater = Heater)
heater = switch_heater_on.parameter("heater")
switch_heater_on.add_precondition(Not(used_heater(heater))) # The heater must not have been already used 
switch_heater_on.add_precondition(Not(is_on(heater)))       # The heater must not be on
switch_heater_on.add_effect(is_warm, True)                  # The temperature becomes warm
switch_heater_on.add_effect(is_on(heater), True)            # The heater switched on
# Define a Variable of type "Heater", used for the existential condition
h_var = Variable("h_var", Heater)
# If an heater is already on, we have an electrical failure
switch_heater_on.add_effect(electrical_failure, True, Exists(is_on(h_var), h_var)) 

switch_heater_off = InstantaneousAction("switch_heater_off", heater = Heater)
heater = switch_heater_off.parameter("heater")
switch_heater_off.add_precondition(is_on(heater))       # The heater must be on
switch_heater_off.add_effect(is_warm, False)            # It is not warm anymore
switch_heater_off.add_effect(is_cold, True)             # It becomes cold
switch_heater_off.add_effect(is_on(heater), False)      # The heater turns off
switch_heater_off.add_effect(used_heater(heater), True) # The heater becomes used

perform_job = InstantaneousAction("perform_job", job = Job)
job = perform_job.parameter("job")
perform_job.add_precondition(is_warm)       # Must be warm to do the job
perform_job.add_effect(is_warm, False)      # It is not warm anymore
perform_job.add_effect(is_cold, True)       # It becomes cold again
perform_job.add_effect(job_done(job), True) # The job is done

# define objects
heater_1 = Object("heater_1", Heater)
heater_2 = Object("heater_2", Heater)

clean = Object("clean", Clean)
work = Object("work", Work)

# define the problem
original_problem = Problem("heaters_and_jobs")
# add the fluents to the problem
original_problem.add_fluent(is_cold, default_initial_value = True)
original_problem.add_fluent(is_warm, default_initial_value = False)
original_problem.add_fluent(electrical_failure, default_initial_value = False)
original_problem.add_fluent(job_done, default_initial_value = False)
original_problem.add_fluent(is_on, default_initial_value = False)
original_problem.add_fluent(used_heater, default_initial_value = False)
# add objects and actions
original_problem.add_objects([heater_1, heater_2, clean, work])
original_problem.add_actions([switch_heater_on, switch_heater_off, perform_job])

# define the problem goals
original_problem.add_goal(Not(electrical_failure))          # No electrical failure
j_var = Variable("j_var", Job)              
original_problem.add_goal(Forall(job_done(j_var), j_var))   # All jobs are done
original_problem.add_goal(Forall(Not(is_on(h_var)), h_var)) # All heaters are switched off

original_problem_kind = original_problem.kind

## Test

To show the usage and the capabilities of the compilers, we will take the problem we just defined and pyperplan, a planner that does not support some features of the original problem. 

With the use of compilers, pyperplan will be able to solve the equivalent compiled problem and then we will rewrite the plan back to be a plan of the original problem.

### Get pyperplan solver

We will now get the pyperplan engine and show that the original problem is not supported.

In [ ]:
with OneshotPlanner(name = "pyperplan") as planner:
    assert not planner.supports(original_problem_kind)

## First compiler: Quantifiers Remover

The quantifiers remover compiles away all the quantifiers.

### How it works

This is done by taking every expression in the problem and grounding the quantifiers (Exists and Forall) with the equivalent formula in the problem. 

For example, in our problem, the goal `Forall(job_done(j_var), j_var)` is equivalent to `job_done(clean) and job_done(work)`, because we only have 2 possible values that the `j_var` variable can have.

### Dimension of expressions created

For every quantifier, a "big" `Or` or `And` will be created. The length of the final expression replacing the quantifier depends on the number of variables of the quantifier itself and the number of objects of each variable type (considering the typing hierarchy).

To get the exact length, we must multiply for every variable the number of objects of the variable type.

For example, in our problem we have quantifiers with only 1 variable, so we get a length of 2 (because we have 2 objects of type `Heater` and 2 objects of type `Job`)

If we had 3 types `A`, `B` and `C`, and 3 objects of type `A`, 2 objects of type `B` and 5 objects of type `C`, with a quantifier with 3 varibales of type `A`, `B` and `C` we would get an expression of length 30. If we have a quantifier with 5 variables, 2 of type `A` and 3 of type `C`, we would get a length of `3^2 * 5^3 = 6 * 125 = 750`; so we see that the length of the expression created increases linearly with the number of objects of the variable type and exponentially with the number of variables in the quantifier.

In [ ]:
from unified_planning.engines import CompilationKind
# The CompilationKind class is defined in the unified_planning/engines/mixins/compiler.py file

# To get the Compiler from the factory we can use the Compiler operation mode.
# It takes a problem_kind and a compilation_kind, and returns a compiler with the capabilities we need
with Compiler(problem_kind = original_problem_kind, 
              compilation_kind = CompilationKind.QUANTIFIERS_REMOVING) as quantifiers_remover:
    # After we have the compiler, we get the compilation result
    qr_result = quantifiers_remover.compile(original_problem, 
                                                              CompilationKind.QUANTIFIERS_REMOVING
                                                             )
    qr_problem = qr_result.problem
    qr_kind = qr_problem.kind
    
    # Check the result of the compilation
    assert original_problem_kind.has_existential_conditions() and original_problem_kind.has_universal_conditions()
    assert not (qr_kind.has_existential_conditions() and qr_kind.has_universal_conditions())
    

## Conditional Effects Remover

The conditional effects remover takes a problem with conditional effects and compiles those away.

### How it works

This is done by taking every action with at least one conditional effect and creating 2 actions: 
- one where the condition of the conditional effect becomes an action precondition and the effect of the conditional effects becomes a normal effect of the action
- one where the inverse of the condition of the conditional effect becomes an action precondition.

### Number of actions created

When an action has more than one conditional effect, let's say `n`, the actions created are `n^2`, because every possible combination of conditions satisfied or unsatisfied must be created and checked.

### Possible Simplifications

The actions created that have conflicting conditions or have no effects are discarded and not added to the resulting problem. 

In [ ]:
# Get the compiler from the factory
with Compiler(problem_kind = qr_kind, 
              compilation_kind = CompilationKind.CONDITIONAL_EFFECTS_REMOVING) as conditional_effects_remover:
    # After we have the compiler, we get the compilation result
    cer_result = conditional_effects_remover.compile(qr_problem, 
                                                     CompilationKind.CONDITIONAL_EFFECTS_REMOVING
                                                     )
    cer_problem = cer_result.problem
    cer_kind = cer_problem.kind
    
    # Check the result of the compilation
    assert original_problem_kind.has_conditional_effects()
    assert qr_kind.has_conditional_effects()
    assert not cer_kind.has_conditional_effects()

## Disjunctive Conditions Remover

The disjunctive conditions remover takes a problem and returns an equivalent problem where every action precondition becomes only a conjunction of terms. Where each term can be a fluent or the negation of a fluent.


### How it works

The disjunctive conditions remover modifies all the actions by making a unique `And` containing all the action's preconditions, computes the equivalent formula of the `And` as a disjunction of conjunctions (an `Or` of `Ands`), and then creates an action for every element of the resulting `Or`. 

Each resulting action has the same effects of the original action and one element of the Or as a precondition.

### Number of actions created

The number of actions created only depends on the length of the resulting `Or`.

For example, if an action precondition is `A and (B or C)`, it will result in `(A and B) or (A and C)`, and this will result in 2 actions, one with precondition `A and B`, and one with precondition `A and C`.

In [ ]:
# Get the compiler from the factory
with Compiler(problem_kind = qr_kind,
              compilation_kind = CompilationKind.DISJUNCTIVE_CONDITIONS_REMOVING) as disjunctive_conditions_remover:
    # After we have the compiler, we get the compilation result
    dcr_result = disjunctive_conditions_remover.compile(cer_problem, 
                                                     CompilationKind.DISJUNCTIVE_CONDITIONS_REMOVING
                                                     )
    dcr_problem = dcr_result.problem
    dcr_kind = dcr_problem.kind
    
    # Check the result of the compilation
    assert qr_kind.has_disjunctive_conditions()
    assert cer_kind.has_disjunctive_conditions()
    assert not dcr_kind.has_disjunctive_conditions()

## Negative Conditions Remover

The negative conditions remover takes a problem with negative conditions and returns an equivalent problem where the `Not` doesn't appear in the action's preconditions or in the problem's goals.

### How it works

For every fluent that appears negated in the conditions or in the goals, a new fluent that represents the same fluent negated is created. 
Every time the original fluent appears negated in an expression, it is replaced with the new fluent.
Every time the original fluent appears in the effect of an action, an effect is added to that action. The added effect assigns the new fluent the negated value assigned to the original fluent; this makes sure that every time the original fluent is modified, also the new fluent is modified with the inverse value, so the new fluent created always represents the opposite of the original fluent.

### Number of fluents and effects created

The number of fluents created depends on the number of fluents that appear negated in the problem's goals or action's preconditions; for each of these fluents, 1 fresh fluent is created.

For every time an original fluent is modified, his negated must be modified too; so the number of effects created is the same to the number of effects on the set of fluents that appear negated in the problem.

In [ ]:
# Get the compiler from the factory
with Compiler(problem_kind = qr_kind,
              compilation_kind = CompilationKind.NEGATIVE_CONDITIONS_REMOVING) as negative_conditions_remover:
    # After we have the compiler, we get the compilation result
    ncr_result = negative_conditions_remover.compile(dcr_problem, 
                                                     CompilationKind.NEGATIVE_CONDITIONS_REMOVING
                                                     )
    ncr_problem = ncr_result.problem
    ncr_kind = ncr_problem.kind
    
    # Check the result of the compilation
    assert original_problem_kind.has_negative_conditions()
    assert qr_kind.has_negative_conditions()
    assert cer_kind.has_negative_conditions()
    assert dcr_kind.has_negative_conditions()
    assert not ncr_kind.has_negative_conditions()

## Solving the obtained problem with pyperplan

After all the compilers have been used in a pipeline, we can solve the problem with pyperplan.

#### Considerations on the plan obtained

As we can see in the simulation, the plan obtained makes no sense for the original plan; but we can focus on the length of the plan. We see it has 6 action instances, intuitively, 3 steps repeated twice: 
- switch one heater on
- get a job done
- switch the heater off

In [ ]:
# Get the planner from the factory
with OneshotPlanner(name = "pyperplan") as planner:
    assert planner.supports(ncr_kind)       # Make sure the planner supports the compiled problem
    ncr_plan = planner.solve(ncr_problem).plan  # Solve the problem and get the plan for the compiled problem
    print(ncr_plan)
    assert len(ncr_plan.actions) == 6

## How to get a plan valid for the original problem

All the compilers we used provide the capabilities of rewriting an action instance of the compiled problem into an action instance of the input problem. 

So, since we used a pipeline of 4 compilers, we have to rewrite back the plan 4 times.

To rewrite back a plan from the compiled problem to the input problem (respectively, compiled_plan and input_plan), we use 2 main features offered by the unified_planning_framework:
- The `CompilationResult.map_back_action_instance`, a field of type: `Callable[[ActionInstance], ActionInstance]`. This function takes an `ActionInstance` of the compiled problem and returns the equivalent `ActionInstance` of the input problem.
- The `Plan.replace_action_instances` method, which takes exactly 1 argument of type `Callable[[ActionInstance], ActionInstance]`, and creates a new plan where every action instance of the original plan is replaced with the result given by the function given as parameter.

Using those 2 features allows us to easily get the equivalent plan for the input problem, and by following the compilers pipeline backwards we can get the plan for the original problem.

In [ ]:
from unified_planning.engines import ValidationResultStatus
# The ValidationResultStatus class is defined in the unified_planning/engines/results.py file

# Create the equivalent plan for the dcr_problem (the one created by the disjunctive conditions remover)
dcr_plan = ncr_plan.replace_action_instances(ncr_result.map_back_action_instance)

# Check to see if the plan is actually valid for the problem
print(dcr_kind)
with PlanValidator(problem_kind = dcr_kind) as validator:
    assert validator.validate(dcr_problem, dcr_plan).status == ValidationResultStatus.VALID

## Final result

Now repeat the process for all the compilers we used.

As we wanted to achieve, with the use of the compilers we managed to solve a problem with pyperplan, when pyperplan was not able to solve said problem.

In [ ]:
# Get the plan for the cer_problem
cer_plan = dcr_plan.replace_action_instances(dcr_result.map_back_action_instance)

# Get the plan for the qr_problem
qr_plan = cer_plan.replace_action_instances(cer_result.map_back_action_instance)

# Get the plan for the original problem
original_plan = qr_plan.replace_action_instances(qr_result.map_back_action_instance)

# Check to see if the obtained plan is actually valid for the original problem
with PlanValidator(problem_kind = original_problem_kind) as validator:
    assert validator.validate(original_problem, original_plan).status == ValidationResultStatus.VALID

print(original_plan)